In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime

import dask
import dask.dataframe as dd

from dask.distributed import Client
from datetime import datetime

# Reduce the number of workers, because we need lots of RAM for loading the CBG shapefile
client = Client(n_workers=2)

In [2]:
# folder = Path("data/reference/census/")
# shapefiles = folder.glob("**/*bg.shp")
# gdf = pd.concat([
#     gpd.read_file(shp)
#     for shp in shapefiles
# ]).pipe(gpd.GeoDataFrame)
# gdf.crs = 'epsg:4269'
# gdf.to_file(folder / 'block_groups.shp')
folder = Path("data/reference/census/")
# shapefiles = folder.glob("**/*bg.shp")
# gdf = pd.concat([
#     gpd.read_file(shp)
#     for shp in shapefiles
# ]).pipe(gpd.GeoDataFrame)
gdf = gpd.read_file(folder / 'block_groups.shp')

In [3]:
df = gdf[['GEOID', 'geometry']]

In [4]:
df['lon'] = df.geometry.apply(lambda g: g.centroid.x)
df['lat'] = df.geometry.apply(lambda g: g.centroid.y)
df = df.drop(['geometry'], axis=1)
df.head()
df.to_parquet("data/output/cbg_centroid.parquet")

/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df = dd.read_parquet("data/output/cbg_centroid.parquet")

In [6]:
def cartesian_product_simplified(left, right):
    la, lb = len(left), len(right)
    ia2, ib2 = np.broadcast_arrays(*np.ogrid[:la,:lb])

    return pd.DataFrame(
        np.column_stack([left.values[ia2.ravel()], right.values[ib2.ravel()]]))

In [ ]:
joined = cartesian_product_simplified(df, df) #.rename(columns={0: 'geoid_orig', 1: 'geom_orig', 2: 'geoid_dest', 3: 'geom_dest'})
# joined['centroid_orig'] = joined['geom_orig'].apply(lambda x: x.centroid)
# joined['centroid_dest'] = joined['geom_dest'].apply(lambda x: x.centroid)
# joined['dist'] = joined.apply(lambda row: row['centroid_orig'].distance(row['centroid_dest']), axis=1)
joined.head()

In [ ]:
joined.to_file(folder / 'block_groups.shp')

In [ ]:
joined[['geoid_orig']]

In [ ]:
folder = Path("data/reference/census/")
# shapefiles = folder.glob("**/*bg.shp")
# gdf = pd.concat([
#     gpd.read_file(shp)
#     for shp in shapefiles
# ]).pipe(gpd.GeoDataFrame)
gdf = gpd.read_file(folder / 'block_groups.shp')

In [ ]:
gdf.head()